# V6: fract2

In [7]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/fract2.router'
netlist_file_path = '../benchmark/fract2.nl'
gridfile_path  = '../benchmark/fract2.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 20
size of layer1: (127, 231)
size of layer2: (127, 231)


In [8]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target，可以显示两层
plot_problem("../output/fract2_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [9]:


def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    if path:               
        for cell in path:
            x, y, _ = cell
            layer1_grid[x][y] = -1

def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty,via_penalty):  
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer:
            cell_cost += via_penalty
        elif (prev_x != x and prev_y != y):  
            cell_cost += bend_penalty
    return cell_cost

def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty,via_penalty):
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 1  # Initial cost for the source cell is 1

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs[current_cell]

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell
                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        wavefront[neighbor_tuple]=cost         

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})     
    neighbors.append({'x': x, 'y': y, 'layer': 3-layer})                                          
    return neighbors

def true_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty,via_penalty):
    nets.sort(key=lambda s: ((s["pin1"]['x']-s["pin2"]['x'])**2+(s["pin1"]['y']-s["pin2"]['y'])**2))
    routing_table = {}
    costs_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        # 防止布线在后续的pin上，先将所有的pin标记为-1；
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
           
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = 1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = 1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = 1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = 1
        print('Routing net:',net_id)
        path,costs = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty,via_penalty) 
        if path is not None:
            mark_path_on_grid(layer1_grid,path)
            routing_table[net_id] = path
            costs_table[net_id] = costs
    routing_table = dict(sorted(routing_table.items(),key=lambda x:x[0]))
    costs_table = dict(sorted(costs_table.items(),key=lambda x:x[0]))
    return routing_table,costs_table

In [10]:
layer1_grid = layer1_grid_original.copy()
layer2_grid = layer2_grid_original.copy()
routing_table,costs_table=true_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty,via_penalty)
plot_path('../output/fract2.jpg',columns=columns,rows=rows,grid1=layer1_grid_original,grid2=layer2_grid_original,path_dict=routing_table) 
generate_output_file(filepath_out,net_num,routing_table)

Routing net: 95
Routing net: 109
Routing net: 119
Routing net: 64
Routing net: 74
Routing net: 105
Routing net: 14
Routing net: 20
Routing net: 22
Routing net: 40
Routing net: 84
Routing net: 12
Routing net: 38
Routing net: 80
Routing net: 42
Routing net: 51
Routing net: 79
Routing net: 103
Routing net: 9
Routing net: 53
Routing net: 91
Routing net: 33
Routing net: 67
Routing net: 117
Routing net: 13
Routing net: 94
Routing net: 101
Routing net: 116
Routing net: 102
Routing net: 4
Routing net: 123
Routing net: 11
Routing net: 37
Routing net: 78
Routing net: 111
Routing net: 113
Routing net: 112
Routing net: 26
Routing net: 107
Routing net: 108
Routing net: 118
Routing net: 81
Routing net: 31
Routing net: 76
Routing net: 100
Routing net: 110
Routing net: 106
Routing net: 115
Routing net: 49
Routing net: 114
Routing net: 44
Routing net: 92
Routing net: 27
Routing net: 10
Routing net: 89
Routing net: 54
Routing net: 63
Routing net: 120
Routing net: 45
Routing net: 65
Routing net: 1
Routin

In [11]:
costs_table

{1: 55,
 2: 65,
 3: 233,
 4: 45,
 5: 260,
 6: 101,
 7: 118,
 8: 125,
 9: 30,
 10: 62,
 11: 44,
 12: 38,
 13: 70,
 14: 30,
 15: 129,
 16: 216,
 17: 163,
 18: 94,
 19: 232,
 20: 30,
 21: 106,
 22: 40,
 23: 201,
 24: 98,
 25: 102,
 26: 38,
 27: 61,
 28: 122,
 29: 78,
 30: 165,
 31: 52,
 32: 125,
 33: 31,
 34: 86,
 35: 147,
 36: 111,
 37: 46,
 38: 38,
 39: 120,
 40: 30,
 41: 150,
 42: 49,
 43: 152,
 44: 60,
 45: 64,
 46: 93,
 47: 73,
 48: 65,
 49: 49,
 50: 67,
 51: 19,
 52: 51,
 53: 30,
 54: 63,
 55: 193,
 56: 185,
 57: 103,
 58: 179,
 59: 99,
 60: 197,
 61: 129,
 62: 150,
 63: 53,
 64: 6,
 65: 87,
 66: 152,
 67: 31,
 68: 125,
 69: 107,
 70: 142,
 71: 129,
 72: 80,
 73: 71,
 74: 6,
 75: 99,
 76: 52,
 77: 84,
 78: 46,
 79: 19,
 80: 47,
 81: 50,
 82: 136,
 83: 190,
 84: 32,
 85: 116,
 86: 143,
 87: 79,
 88: 102,
 89: 62,
 90: 224,
 91: 30,
 92: 70,
 93: 132,
 94: 32,
 95: 4,
 96: 117,
 97: 119,
 98: 129,
 99: 88,
 100: 53,
 101: 42,
 102: 43,
 103: 19,
 104: 87,
 105: 6,
 106: 55,
 107: 48,


In [12]:
cnt = 0
for item in routing_table.keys():
    if routing_table[item]:
        cnt += 1
cnt

125